https://www.itread01.com/content/1547127786.html

#### 如何創建自己的 Dataloader
https://blog.csdn.net/york1996/article/details/84141034

### Pytorch Dataloader 參數意義：
* dataset，这个就是PyTorch已有的数据读取接口（比如torchvision.datasets.ImageFolder）或者自定义的数据接口的输出，该输出要么是torch.utils.data.Dataset类的对象，要么是继承自torch.utils.data.Dataset类的自定义类的对象。
* batch_size，根据具体情况设置即可。
* shuffle，一般在训练数据中会采用。
* collate_fn，是用来处理不同情况下的输入dataset的封装，一般采用默认即可，除非你自定义的数据读取输出非常少见。
* batch_sampler，从注释可以看出，其和batch_size、shuffle等参数是互斥的，一般采用默认。
* sampler，从代码可以看出，其和shuffle是互斥的，一般默认即可。
* num_workers，从注释可以看出这个参数必须大于等于0，0的话表示数据导入在主进程中进行，其他大于0的数表示通过多个进程来导入数据，可以加快数据导入速度。
* pin_memory，注释写得很清楚了： pin_memory (bool, optional): If True, the data loader will copy tensors into CUDA pinned memory before returning them. 也就是一个数据拷贝的问题。
* timeout，是用来设置数据读取的超时时间的，但超过这个时间还没读取到数据的话就会报错。

In [61]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils import data
import tensorflow as tf
import random
import numpy as np
random.seed(1461)

In [69]:
BATCH_SIZE = 5

x = np.random.randint(0, 9, size=(10, 2))
x = torch.Tensor(x)
y = torch.linspace(9, 0, 10)
print(f'x: {x}')
print(f'y: {y}')

x: tensor([[5., 1.],
        [3., 2.],
        [0., 7.],
        [2., 8.],
        [5., 7.],
        [7., 4.],
        [2., 0.],
        [2., 2.],
        [2., 8.],
        [7., 8.]])
y: tensor([9., 8., 7., 6., 5., 4., 3., 2., 1., 0.])


In [70]:
'''先轉換成 torch 能事別的 Dataset '''
torch_dataset = data.TensorDataset(x, y)

In [47]:
''' 觀察資料型別 '''
type(torch_dataset)

torch.utils.data.dataset.TensorDataset

In [48]:
type(x)

torch.Tensor

In [73]:
''' 把 dataset 放入 DataLoader '''
loader = data.DataLoader(
    dataset = torch_dataset,
    batch_size= BATCH_SIZE,
    shuffle= True,
    num_workers= 8,
    drop_last= True
)


In [50]:
for epoch in range(5):
    i = 0
    for batch_x, batch_y in loader:
        i = i + 1
        print('Epoch: {} | num: {} | batch_x: {} | batch_y: {}'
              .format(epoch, i, batch_x, batch_y))
        

Epoch: 0 | num: 1 | batch_x: tensor([8., 9., 4., 3., 5.]) | batch_y: tensor([1., 0., 5., 6., 4.])
Epoch: 0 | num: 2 | batch_x: tensor([7., 2., 0., 6., 1.]) | batch_y: tensor([2., 7., 9., 3., 8.])
Epoch: 1 | num: 1 | batch_x: tensor([6., 2., 4., 1., 7.]) | batch_y: tensor([3., 7., 5., 8., 2.])
Epoch: 1 | num: 2 | batch_x: tensor([3., 5., 0., 8., 9.]) | batch_y: tensor([6., 4., 9., 1., 0.])
Epoch: 2 | num: 1 | batch_x: tensor([2., 6., 3., 4., 5.]) | batch_y: tensor([7., 3., 6., 5., 4.])
Epoch: 2 | num: 2 | batch_x: tensor([7., 1., 0., 9., 8.]) | batch_y: tensor([2., 8., 9., 0., 1.])
Epoch: 3 | num: 1 | batch_x: tensor([6., 7., 2., 4., 1.]) | batch_y: tensor([3., 2., 7., 5., 8.])
Epoch: 3 | num: 2 | batch_x: tensor([0., 5., 9., 3., 8.]) | batch_y: tensor([9., 4., 0., 6., 1.])
Epoch: 4 | num: 1 | batch_x: tensor([1., 5., 7., 3., 2.]) | batch_y: tensor([8., 4., 2., 6., 7.])
Epoch: 4 | num: 2 | batch_x: tensor([9., 8., 6., 0., 4.]) | batch_y: tensor([0., 1., 3., 9., 5.])


In [74]:
for epoch in range(3):
    i = 0
    ''' 每一步 loader 釋放一小批數據用來學習 '''
    for step, (batch_x, batch_y) in enumerate(loader):
        ''' 假設這是我們訓練的地方... '''
        i = i + 1
        ''' 列印一些數據 '''
        print('Epoch: {} | num: {} | batch_x: {} | batch_y: {}'
              .format(epoch, i, batch_x, batch_y))

Epoch: 0 | num: 1 | batch_x: tensor([[2., 2.],
        [2., 8.],
        [7., 4.],
        [5., 1.],
        [3., 2.]]) | batch_y: tensor([2., 1., 4., 9., 8.])
Epoch: 0 | num: 2 | batch_x: tensor([[2., 8.],
        [5., 7.],
        [2., 0.],
        [7., 8.],
        [0., 7.]]) | batch_y: tensor([6., 5., 3., 0., 7.])
Epoch: 1 | num: 1 | batch_x: tensor([[5., 7.],
        [7., 8.],
        [7., 4.],
        [2., 8.],
        [5., 1.]]) | batch_y: tensor([5., 0., 4., 6., 9.])
Epoch: 1 | num: 2 | batch_x: tensor([[2., 8.],
        [0., 7.],
        [2., 0.],
        [2., 2.],
        [3., 2.]]) | batch_y: tensor([1., 7., 3., 2., 8.])
Epoch: 2 | num: 1 | batch_x: tensor([[2., 8.],
        [0., 7.],
        [2., 0.],
        [2., 8.],
        [7., 4.]]) | batch_y: tensor([1., 7., 3., 6., 4.])
Epoch: 2 | num: 2 | batch_x: tensor([[5., 7.],
        [3., 2.],
        [2., 2.],
        [5., 1.],
        [7., 8.]]) | batch_y: tensor([5., 8., 2., 9., 0.])


---

https://stackoverflow.com/questions/62055185/print-the-values-of-torch-data-dataset-object

''' 如何列印出 Dataloader 中的內容 '''
```print(list(loader))```